In [4]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce

In [5]:
def get(q):
    return pd.DataFrame(sql.execute(q))

In [6]:
q='''select distinct name, count(*) as total_alerts from project
    join alert a on project.id = a.project_id
    where language in ('C','C++')
    and is_invalid=0
    group by name; '''
ta=get(q)
ta

,name,total_alerts
0,Kodi,2393
1,Firefox,12944
2,Linux,19514
3,Samba,4507
4,!CHAOS Control System,1980
5,Chromium EC,964
6,OpenCV,2443
7,LibreOffice,11982
8,Thunderbird,885
9,VTK,2522


In [7]:
q='''select name, count(*) as total_scans from project
join snapshot s on project.id = s.project_id
group by name;'''
ts=get(q)
ts

,name,total_scans
0,!CHAOS Control System,170
1,Chromium EC,1713
2,Firefox,662
3,Kodi,489
4,LibreOffice,414
5,Linux,648
6,OpenCV,553
7,ovirt-engine,835
8,Samba,770
9,Thunderbird,438


In [8]:
q='''select name, timestampdiff(year, start_date,end_date) as "time period (years)"
from project;'''
span=get(q)
span

,name,time period (years)
0,Kodi,7
1,ovirt-engine,6
2,Firefox,12
3,Linux,8
4,Samba,14
5,!CHAOS Control System,4
6,Chromium EC,4
7,OpenCV,7
8,LibreOffice,7
9,Thunderbird,13


In [9]:
q='''select name, datediff(s1.date,s2.date) as "scan interval (days)"
        from snapshot s1
            join snapshot s2 on s1.last_snapshot=s2.id and s2.project_id=s1.project_id
        join project p on s1.project_id = p.id;'''
intvl=get(q)
intvl=intvl.groupby(intvl.name)[['scan interval (days)']].median()
intvl

,scan interval (days)
name,
!CHAOS Control System,1
Chromium EC,0
Firefox,2
Kodi,3
LibreOffice,4
Linux,3
OpenCV,2
Samba,3
Thunderbird,1


In [10]:
df=reduce(lambda x,y: pd.merge(x,y,on='name'),[ta,ts,span,intvl])
df = df.sort_values('total_alerts', ascending=False)
df

,name,total_alerts,total_scans,time period (years),scan interval (days)
2,Linux,19514,648,8,3
1,Firefox,12944,662,12,2
7,LibreOffice,11982,414,7,4
3,Samba,4507,770,14,3
9,VTK,2522,74,1,7
6,OpenCV,2443,553,7,2
0,Kodi,2393,489,7,3
4,!CHAOS Control System,1980,170,4,1
5,Chromium EC,964,1713,4,0
8,Thunderbird,885,438,13,1


In [11]:
q='select name, start_date, end_date from project'
temp= get(q)
df=reduce(lambda x,y: pd.merge(x,y,on='name'),[df,temp])


In [12]:
df.to_csv ('project_info_C_C++.csv', index = False, header=True)

In [13]:
q='''select name, count(*) as triaged from alert a
join project p on a.project_id = p.id
where is_invalid=0 and language in ('C','C++')
and last_triaged is not null
group by project_id;'''
temp=get(q)
df=reduce(lambda x,y: pd.merge(x,y,on='name'),[df,temp])
df.triaged = round(df.triaged/df.total_alerts *100,2)
df

,name,total_alerts,total_scans,time period (years),scan interval (days),start_date,end_date,triaged
0,Linux,19514,648,8,3,2012-05-17 00:00:00,2020-06-15 23:27:15,13.66
1,Firefox,12944,662,12,2,2006-02-22 00:00:00,2018-10-27 07:23:51,36.33
2,LibreOffice,11982,414,7,4,2012-10-13 16:29:06,2020-06-26 22:28:07,55.42
3,Samba,4507,770,14,3,2006-02-23 00:00:00,2020-06-17 09:05:31,52.58
4,VTK,2522,74,1,7,2015-12-21 12:23:30,2017-08-12 00:21:28,2.50
5,OpenCV,2443,553,7,2,2012-12-12 03:05:09,2020-06-30 21:54:52,9.41
6,Kodi,2393,489,7,3,2012-08-28 00:00:00,2020-05-16 07:42:09,42.42
7,!CHAOS Control System,1980,170,4,1,2015-01-14 03:22:14,2019-03-19 02:39:22,12.37
8,Chromium EC,964,1713,4,0,2016-01-15 17:37:41,2020-07-06 07:37:56,28.11
9,Thunderbird,885,438,13,1,2006-04-12 17:23:58,2020-02-29 20:18:19,27.23
